In [7]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from tqdm.notebook import tqdm
import time
import re
import requests
import random
import pickle
import openpyxl

https://manyda.tistory.com/10
여기에서 나온 거 활용

In [3]:
#Queue의 기본적인 기능 구현
class Queue():
    def __init__(self, maxsize):
        self.queue = []
        self.maxsize = maxsize
        
    # Queue에 Data 넣음
    def enqueue(self, data):
        self.queue.append(data)

    # Queue에 가장 먼저 들어온 Data 내보냄
    def dequeue(self):
        dequeue_object = None
        if self.isEmpty():
            print("Queue is Empty")
        else:
            dequeue_object = self.queue[0]
            self.queue = self.queue[1:]
        return dequeue_object
    
    # Queue에 가장 먼저들어온 Data return
    def peek(self):
        peek_object = None
        if self.isEmpty():
            print("Queue is Empty")
        else:
            peek_object = self.queue[0]
        return peek_object
    
    # Queue가 비어있는지 확인
    def isEmpty(self):
        is_empty = False
        if len(self.queue) == 0:
            is_empty = True
        return is_empty
    
    # Queue의 Size가 Max Size를 초과하는지 확인
    def isMaxSizeOver(self):
        queue_size = len(self.queue)
        if (queue_size > self.maxsize):
            return False
        else :
            return True

In [27]:
if __name__=="__main__":
    #set option of selenium
    options = webdriver.ChromeOptions()
    options.add_argument('window-size=1920x1080')
    options.add_argument('disable-gpu')
    options.add_argument('user')
    options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
    options.add_argument("lang=ko_KR")
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

    #target of crawling
    data_list = []
    driver.get("https://www.youtube.com/watch?v=QndOyQtTHUQ")
    
    #페이지 Open 후 기다리는 시간
    time.sleep(15.0)

    #down the scroll
    body = driver.find_element_by_tag_name('body')
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    # max size 50의 Queue 생성
    # 0.1sec * 50 = 5sec 동안 Scroll 업데이트가 없으면 스크롤 내리기 종료
    szQ = Queue(50)
    enqueue_count = 0
    
    while True:
        # Scroll 내리기
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        
        # Scroll Height를 가져오는 주기
        time.sleep(0.1)
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        # Queue가 꽉 차는 경우 스크롤 내리기 종료
        if(enqueue_count > szQ.maxsize):
            break
        
        # 첫 Loop 수행 (Queue가 비어있는 경우) 예외 처리
        if(szQ.isEmpty()) :
            szQ.enqueue(new_page_height)
            enqueue_count += 1
            
        # Queue에 가장 먼저 들어온 데이터와 새로 업데이트 된 Scroll Height를 비교함
        # 같으면 그대로 Enqueue, 다르면 Queue의 모든 Data를 Dequeue 후 새로운 Scroll Height를 Enqueue 함.    
        else :
            if(szQ.peek() == new_page_height) :
                szQ.enqueue(new_page_height)
                enqueue_count += 1
            else :
                szQ.enqueue(new_page_height)
                for z in range(enqueue_count) :
                    szQ.dequeue()
                enqueue_count = 1
        
        # 기존의 Scroll 내리는 방식      
        #if new_page_height == last_page_height:
        #    break
        #last_page_height = new_page_height
        #time.sleep(2.0)
    
    # print ("[PASS] Get all comments of URL")

    html0 = driver.page_source
    driver.close()
    html = BeautifulSoup(html0, 'html.parser')

    comments_list = html.findAll('ytd-comment-thread-renderer', {'class':'style-scope ytd-item-section-renderer'})
    # print (comments_list)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\Huiseop\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
C:\Users\Huiseop\AppData\Local\Temp\ipykernel_25524\4004985141.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


In [28]:
for j in range(len(comments_list)):
     #contents of comment
        comment = comments_list[j].find('yt-formatted-string',{'id':'content-text'}).text
        comment = comment.replace('\n', '') 
        comment = comment.replace('\t', '')
        #print(comment) 
        youtube_id = comments_list[j].find('a', {'id': 'author-text'}).span.text
        youtube_id = youtube_id.replace('\n', '') 
        youtube_id = youtube_id.replace('\t', '') 
        youtube_id = youtube_id.strip()
    
        raw_date = comments_list[j].find('yt-formatted-string', { 'class': 'published-time-text above-comment style-scope ytd-comment-renderer'})
        date = raw_date.a.text
    
        try:
            like_num = comments_list[j].find('span', {'id': 'vote-count-middle', 'class': 'style-scope ytd-comment-action-buttons-renderer', 'aria-label': re.compile('좋아요')}).text
            like_num = like_num.replace('\n', '') 
            like_num = like_num.replace('\t', '')
            like_num = like_num.strip()
        except: like_num = 0
    
        data = {'youtube_id': youtube_id, 'comment': comment, 'date': date, 'like_num': like_num}
        data_list.append(data)
    
result_df = pd.DataFrame(data_list, columns=['youtube_id','comment','date','like_num'])    
result_df.to_excel("./data.xlsx", index = False)

https://bit.ly/3ymqjH5
여기에서 나온거 활용

In [32]:
# 셀레니움 옵션 설정
options = webdriver.ChromeOptions()
# options.add_argument('headless') # 크롬 띄우는 창 없애기
options.add_argument('window-size=1920x1080') # 크롬드라이버 창크기
# options.add_argument("disable-gpu") #그래픽 성능 낮춰서 크롤링 성능 쪼금 높이기
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36") # 네트워크 설정
options.add_argument("lang=ko_KR") # 사이트 주언어
driver = webdriver.Chrome(ChromeDriverManager().install(),
    chrome_options=options)

# 크롤링 목표 : 해당 영상에 대한 댓글 id, 댓글 내용, 댓글의 좋아요 개수, 날짜 추출
data_list = []
driver.get("https://www.youtube.com/watch?v=AeVP4dzq174&list=PLVQ788SqBYulpfcD8iKxx2lqEYuCX8aZh&ab_channel=%EC%88%8F%EB%B0%95%EC%8A%A4")

# 스크롤 내리기
body = driver.find_element_by_tag_name('body')
last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(2)
    new_page_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_page_height == last_page_height:
        break
    
# bs4 html 파싱
html0 = driver.page_source
html = BeautifulSoup(html0, 'html.parser')
comments_list = html.findAll('ytd-comment-thread-renderer', {'class': 'style-scope ytd-item-section-renderer'})

for j in range(len(comments_list)):
    ## 재생목록이름
    list_name = comments_list[j].find('a', {'class': 'yt-simple-endpoint style-scope yt-formatted-string'}).text

    ## 전체 조회수
    views = comments_list[j].find('span', {'class': 'view-count style-scope ytd-video-view-count-renderer'}).text

    ## 제목 
    name = comments_list[j].find('yt-formatted-string', {'class': 'style-scope ytd-video-primary-info-renderer'}).text
    
    ## 댓글 내용
    comment = comments_list[j].find('yt-formatted-string', {'id': 'content-text'}).text
    comment = comment.replace('\n', '') # 줄 바뀜 없애기
    comment = comment.replace('\t', '') # 탭 줄이기
    # print(comment)
    
    ## 댓글 좋아요 개수 (0인 경우 예외 처리)
    try:
        like_num = comments_list[j].find('span',
                                     {'id': 'vote-count-middle',
                                      'class': 'style-scope ytd-comment-action-buttons-renderer',
                                      'aria-label': re.compile('좋아요')}).text
        like_num = like_num.replace('\n', '') # 줄 바뀜 없애기
        like_num = like_num.replace('\t', '') # 탭 줄이기
        like_num = like_num.strip()
    except:
        like_num = 0

    data = {'list_name': list_name, 'name': name, 'views': views, 'comment': comment, 'like_num': like_num}
    data_list.append(data)
    # print(data)

result_df = pd.DataFrame(data_list,
                         columns=['list_name','name','views','comment', 'like_num'])
result_df.to_excel("./data1.xlsx", index = False)
driver.close()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\Huiseop\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
C:\Users\Huiseop\AppData\Local\Temp\ipykernel_25524\1205492189.py:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(),


WebDriverException: Message: chrome not reachable
  (Session info: chrome=103.0.5060.66)
